### **Data Reading**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@retailsalesanalysissanil.dfs.core.windows.net/products")
df.display()

### **Functions**

### Applying function using SQL 

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
create or replace function rsa_cata.bronze.discount_func(p_price double)
returns double
language sql
return p_price * 0.90

In [0]:
%sql
select product_id, price, rsa_cata.bronze.discount_func(price) as discounted_price
from products

### Applying function using pyspark

In [0]:
df.withColumn("discounted_price", expr("rsa_cata.bronze.discount_func(price)")).display()

### Creating function using Python language

In [0]:
%sql
create or replace function rsa_cata.bronze.upper_func(p_brand string)
returns string
language python
as
$$
  return p_brand.upper()
$$

In [0]:
%sql
select product_id, brand, rsa_cata.bronze.upper_func(brand) as brand_upper
from products

### **Data Writing**

In [0]:
df.write.format("delta")\
    .mode("append")\
    .save("abfss://silver@retailsalesanalysissanil.dfs.core.windows.net/products")

### **Table Creation to top of the Data**

In [0]:
%sql
create table if not exists rsa_cata.silver.products_silver 
using delta
location 'abfss://silver@retailsalesanalysissanil.dfs.core.windows.net/products'